In [32]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from torch_geometric.utils import softmax
import torch_geometric.utils.convert as conv
import torch
from torch_geometric.nn import GCNConv, GATConv,SAGEConv
import torch.nn.functional as F
from torch.nn.functional import softmax

# load the graph from the graphml file
def load_graph(file_path):
    G = nx.read_graphml(file_path)
    return G

file_path = 'data/pubmed-diabetes.graphml'
# define the classification label
classification_label = 'label'


In [21]:
G = load_graph(file_path)

In [28]:
#Removing not useful attributes
for s,d in G.edges:
    del G.edges[s,d]['id']

In [29]:
print(G, type(G))

print(list(G.nodes(data=True))[0])

from collections import Counter

labels = []
for s,d in G.edges:
    labels.append(G.edges[s,d]['label'])
print(Counter(labels))

DiGraph with 19717 nodes and 44338 edges <class 'networkx.classes.digraph.DiGraph'>
('19127292', {'w-rat': 0.0, 'w-common': 0.0, 'w-use': 0.0, 'w-examin': 0.02889892370027422, 'w-pathogenesi': 0.0, 'w-retinopathi': 0.0, 'w-mous': 0.0, 'w-studi': 0.03927472173927795, 'w-anim': 0.0, 'w-model': 0.0, 'w-metabol': 0.0, 'w-abnorm': 0.0, 'w-contribut': 0.0, 'w-develop': 0.0, 'w-investig': 0.0, 'w-mice': 0.0, 'w-2': 0.0051138308462225415, 'w-month': 0.0, 'w-compar': 0.017653254069202904, 'w-obtain': 0.0, 'w-method': 0.0, 'w-induc': 0.0, 'w-6': 0.0, 'w-inject': 0.0, 'w-experiment': 0.0, 'w-normal': 0.0, 'w-diet': 0.0, 'w-30': 0.0, 'w-hyperglycemia': 0.0, 'w-level': 0.0, 'w-lipid': 0.0, 'w-oxid': 0.0, 'w-activ': 0.0, 'w-protein': 0.0, 'w-kinas': 0.0, 'w-c': 0.0, 'w-measur': 0.011734827227009909, 'w-result': 0.005626481865792995, 'w-increas': 0.013080628861391524, 'w-retin': 0.0, 'w-stress': 0.0, 'w-3': 0.009410652924576506, 'w-similar': 0.0, 'w-observ': 0.013638082547296626, 'w-conclus': 0.00959

KeyError: 'label'

In [30]:
edge = list(G.edges(data=True))[10]
print(edge)

('19127292', '8886555', {})


Convert the node attributes into integers

In [41]:


labels = {1:0,2:1,3:2}

g_node_attrs = list(list(G.nodes(data=True))[0][1].keys())

print(g_node_attrs)

classes = set()
for id in G.nodes:
    classes.add(G.nodes[id][classification_label])
for id in G.nodes:
    G.nodes[id][classification_label] = labels[G.nodes[id][classification_label]]


num_classes = len(classes)

print(classes)

w_attrs = [attr for attr in g_node_attrs if 'w_' in attr]

print(len(g_node_attrs))

['w-rat', 'w-common', 'w-use', 'w-examin', 'w-pathogenesi', 'w-retinopathi', 'w-mous', 'w-studi', 'w-anim', 'w-model', 'w-metabol', 'w-abnorm', 'w-contribut', 'w-develop', 'w-investig', 'w-mice', 'w-2', 'w-month', 'w-compar', 'w-obtain', 'w-method', 'w-induc', 'w-6', 'w-inject', 'w-experiment', 'w-normal', 'w-diet', 'w-30', 'w-hyperglycemia', 'w-level', 'w-lipid', 'w-oxid', 'w-activ', 'w-protein', 'w-kinas', 'w-c', 'w-measur', 'w-result', 'w-increas', 'w-retin', 'w-stress', 'w-3', 'w-similar', 'w-observ', 'w-conclus', 'w-play', 'w-import', 'w-role', 'w-present', 'w-p', 'w-m', 'w-r', 'w-muscl', 'w-control', 'w-chang', 'w-dure', 'w-lower', 'w-higher', 'w-mass', 'w-correl', 'w-decreas', 'w-determin', 'w-concentr', 'w-stimul', 'w-period', 'w-caus', 'w-mark', 'w-group', 'w-evid', 'w-fast', 'w-type', 'w-signific', 'w-differ', 'w-ratio', 'w-suggest', 'w-degre', 'w-occur', 'w-vivo', 'w-respect', 'w-dysfunct', 'w-region', 'w-high', 'w-appear', 'w-sever', 'w-affect', 'w-cardiovascular', 'w-compl

In [42]:
# delete all the graph attributes, since they are not useful for node classification

# print(type(node_attrs))
# print(edge_attrs)

# graph.graph['node_default'] = True
# graph.graph['edge_default'] = True

print(G.graph, type(G.graph))

if 'node_default' in G.graph:
    del G.graph['node_default']
if 'edge_default' in G.graph:
    del G.graph['edge_default']

if 'edge_index' in G.graph:
    del G.graph['edge_index']

attrs = [attr for attr, _ in G.graph.items()]

for attr in attrs:
    del G.graph[attr]

print(G.graph, type(G.graph))

{} <class 'dict'>
{} <class 'dict'>


In [43]:
data = conv.from_networkx(G, group_node_attrs=g_node_attrs)

print(f'Number of edges: {data.num_edges}')

print(data, type(data))

Number of edges: 44338
Data(edge_index=[2, 44338], x=[19717, 501]) <class 'torch_geometric.data.data.Data'>


Extract the classification label from the nodes attributes

In [44]:
print(data.x)

# shuffle the data by row
data.x = data.x[torch.randperm(data.x.size()[0])]
print(data.x.shape)

# get the classification attibute from x and assign it to y
data.y = data.x[:, -1]
# data.y = torch.stack([y[0] for y in data.y])

# remove the classification attribute from x
data.x = data.x[:, :-1].float()

print(data.x)
print(data.y)

print(data)

tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 2.0000],
        [0.0000, 0.0000, 0.0088,  ..., 0.0000, 0.0000, 2.0000],
        [0.0000, 0.0000, 0.0178,  ..., 0.0000, 0.0000, 2.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 2.0000],
        [0.0000, 0.0000, 0.0188,  ..., 0.0000, 0.0000, 1.0000],
        [0.0000, 0.0000, 0.0064,  ..., 0.0000, 0.0000, 2.0000]])
torch.Size([19717, 501])
tensor([[0.0000, 0.0000, 0.0323,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0180,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0486, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])
tensor([2., 1., 2.,  ..., 1., 2., 2.])
Data(edge_index=[2, 44338], x=[19717, 500], y=[19717])


Define training, validation and test sets randomly

In [45]:
from torch_geometric.transforms import RandomNodeSplit

# split the data in train, validation and test sets
# it adds to data train, val and test masks
rns = RandomNodeSplit('random', num_val=0.1, num_test=0.2)

data = rns(data)

print(data)

Data(edge_index=[2, 44338], x=[19717, 500], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717])


One-hot encode the classification labels

In [46]:
# one hot encode labels
data.y = torch.nn.functional.one_hot(data.y.long(), num_classes=num_classes).to(torch.float)

print(data.y)
print(data)

tensor([[0., 0., 1.],
        [0., 1., 0.],
        [0., 0., 1.],
        ...,
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 0., 1.]])
Data(edge_index=[2, 44338], x=[19717, 500], y=[19717, 3], train_mask=[19717], val_mask=[19717], test_mask=[19717])


In [47]:
print(data.x.dtype)
print(data.y.dtype)
print(data.edge_index.dtype)

torch.float32
torch.float32
torch.int64


In [48]:
print(list(data.edge_index))

[tensor([    0,     0,     0,  ..., 19280, 19523, 19681]), tensor([    1,  4530,  5212,  ...,  6030,  1043, 18742])]


In [49]:
from sklearn.manifold import TSNE

def visualize(h, color):
    z = TSNE(n_components=2).fit_transform(h.detach().cpu().numpy())

    plt.figure(figsize=(10,10))
    plt.xticks([])
    plt.yticks([])

    plt.scatter(z[:, 0], z[:, 1], s=70, c=color, cmap="Set2")
    plt.show()

In [50]:
class GCN(torch.nn.Module):
    def __init__(self, input_features, hidden_channels):
        super().__init__()
        torch.manual_seed(1234567)
        self.conv1 = GCNConv(input_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, num_classes)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.4, training=self.training)
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.4, training=self.training)
        # x = softmax(x)
        # x = F.log_softmax(x, dim=1)
        x = F.softmax(x, dim=1)
        return x

in_features = data.x.shape[1]

model = GCN(input_features=in_features, hidden_channels=16)
model.eval()
print(model)

out = model(data.x, data.edge_index)
#visualize(out, color=data.y)
print(out)

model = GCN(input_features=in_features, hidden_channels=16)
optimizer = torch.optim.Adam(model.parameters(), lr=0.1, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()


GCN(
  (conv1): GCNConv(500, 16)
  (conv2): GCNConv(16, 3)
)
tensor([[0.3333, 0.3333, 0.3333],
        [0.3319, 0.3335, 0.3346],
        [0.3328, 0.3345, 0.3328],
        ...,
        [0.3260, 0.3403, 0.3337],
        [0.3315, 0.3322, 0.3363],
        [0.3364, 0.3322, 0.3315]], grad_fn=<SoftmaxBackward0>)


In [53]:
class SAGE(torch.nn.Module):
    def __init__(self, input_features, hidden_channels):
        super().__init__()
        torch.manual_seed(1234567)
        self.conv1 = SAGEConv(input_features, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, num_classes)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.4, training=self.training)
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.4, training=self.training)
        x = F.softmax(x, dim=1)
        return x

in_features = data.x.shape[1]

model = SAGE(input_features=in_features, hidden_channels=16)
optimizer = torch.optim.Adam(model.parameters(), lr=0.1, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()


In [45]:
class GAT(torch.nn.Module):
    def __init__(self, hidden_channels, heads):
        super().__init__()
        torch.manual_seed(1234567)

        self.hid = 8
        self.in_head = 8
        self.out_head = 1

        self.conv1 = GATConv(in_features, self.hid, heads=self.in_head, dropout=0.6)
        self.conv2 = GATConv(self.hid*self.in_head, num_classes, concat=False,
                             heads=self.out_head, dropout=0.6)


    def forward(self, x, edge_index):

        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)

        return F.softmax(x, dim=1)

model = GAT(hidden_channels=8, heads=8)
print(model)

optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

GAT(
  (conv1): GATConv(3703, 8, heads=8)
  (conv2): GATConv(64, 6, heads=1)
)


In [54]:
def train():
      model.train()
      optimizer.zero_grad()  # Clear gradients.
      out = model(data.x, data.edge_index)  # Perform a single forward pass.
      loss = criterion(out[data.train_mask], data.y[data.train_mask])
      val_loss = criterion(out[data.val_mask], data.y[data.val_mask])  # Compute the loss solely based on the training nodes.
      loss.backward()  # Derive gradients.
      optimizer.step()  # Update parameters based on gradients.
      return loss, val_loss


def test(models, ds='test'):
      models.eval()
      out = models(data.x, data.edge_index)
      pred = out.argmax(dim=1)  # Use the class with highest probability.
      if ds == 'test':
            outs = out[data.test_mask]
            mask = data.test_mask
            d_set = pred[data.test_mask]
            label_set = data.y[data.test_mask]
      elif ds == 'val':
            outs = out[data.val_mask]
            mask = data.val_mask
            d_set = pred[data.val_mask]
            label_set = data.y[data.val_mask]
      else:
            outs = out[data.train_mask]
            mask = data.train_mask
            d_set = pred[data.train_mask]
            label_set = data.y[data.train_mask]
      # print(outs)
      # print(label_set)
      label_set = label_set.argmax(dim=1)
      test_correct = d_set == label_set  # Check against ground-truth labels.
      test_acc = int(test_correct.sum()) / int(mask.sum())  # Derive ratio of correct predictions.
      return test_acc

In [55]:
start_patience = 40
patience = start_patience
best_val_loss = 10000
best_model = model

for epoch in range(1, 101):
    if patience == 0:
        break
    loss, val_loss = train()
    if val_loss <= best_val_loss:
        patience = start_patience
        best_val_loss = val_loss
        best_model = model
    else:
        patience -= 1

    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val Loss: {val_loss:.4f}')


train_acc = test(best_model, 'train')
val_acc = test(best_model, 'val')
test_acc = test(best_model)

print(f'Train Accuracy: {train_acc:.4f}')
print(f'Val Accuracy: {val_acc:.4f}')
print(f'Test Accuracy: {test_acc:.4f}')

model.eval()

# out = model(data.x, data.edge_index)
# visualize(out, color=data.y)

Epoch: 001, Loss: 1.1065, Val Loss: 1.1086
Epoch: 002, Loss: 1.0848, Val Loss: 1.0941
Epoch: 003, Loss: 1.0495, Val Loss: 1.0927
Epoch: 004, Loss: 1.0034, Val Loss: 1.0866
Epoch: 005, Loss: 1.0200, Val Loss: 1.0882
Epoch: 006, Loss: 0.9548, Val Loss: 1.0834
Epoch: 007, Loss: 0.9216, Val Loss: 1.0909
Epoch: 008, Loss: 0.9549, Val Loss: 1.0811
Epoch: 009, Loss: 1.0102, Val Loss: 1.0757
Epoch: 010, Loss: 0.9151, Val Loss: 1.0740
Epoch: 011, Loss: 0.9079, Val Loss: 1.0669
Epoch: 012, Loss: 0.8894, Val Loss: 1.0638
Epoch: 013, Loss: 0.9315, Val Loss: 1.0724
Epoch: 014, Loss: 0.9266, Val Loss: 1.0612
Epoch: 015, Loss: 0.9370, Val Loss: 1.0593
Epoch: 016, Loss: 0.9242, Val Loss: 1.0549
Epoch: 017, Loss: 0.9405, Val Loss: 1.0635
Epoch: 018, Loss: 0.9344, Val Loss: 1.0616
Epoch: 019, Loss: 0.9445, Val Loss: 1.0556
Epoch: 020, Loss: 0.8803, Val Loss: 1.0628
Epoch: 021, Loss: 0.8895, Val Loss: 1.0671
Epoch: 022, Loss: 0.9447, Val Loss: 1.0669
Epoch: 023, Loss: 0.8951, Val Loss: 1.0600
Epoch: 024,

SAGE(
  (conv1): SAGEConv(500, 16, aggr=mean)
  (conv2): SAGEConv(16, 3, aggr=mean)
)